<a href="https://colab.research.google.com/github/TasmeerJamali/AgenticAI_and_Complete_Initial_Python-Codes/blob/main/Copy_of_OPENAI_SDK_OPENROUTER1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [ ]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [ ]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "mistralai/mistral-small-24b-instruct-2501:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [ ]:
BASE_URL

'https://openrouter.ai/api/v1'

In [ ]:
import requests
import json

response = requests.post(
  url=f"https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "Hi, i have 1000 pkr i want you to convert into usd"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745143163-DRxNgsat72SUUpmYe7Lq', 'provider': 'Chutes', 'model': 'mistralai/mistral-small-24b-instruct-2501', 'object': 'chat.completion', 'created': 1745143163, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'To convert Pakistani Rupees (PKR) to US Dollars (USD), we need the current exchange rate. As of my last update in October 2023, I don\'t have real-time data, but I can guide you on how to find the current exchange rate.\n\n1. **Check a Financial Website**: Visit a reliable financial website like XE.com, OANDA, or a similar service.\n2. **Use a Currency Conversion App**: There are many apps available for both iOS and Android that provide real-time exchange rates.\n3. **Bank or Financial Institution**: Your bank or a financial institution can provide the current exchange rate.\n4. **Google**: Simply search "1000 PKR to USD" on Google, and it will show the current exchange ra

In [ ]:
data = response.json()
data['choices'][0]['message']['content']

'To convert Pakistani Rupees (PKR) to US Dollars (USD), we need the current exchange rate. As of my last update in October 2023, I don\'t have real-time data, but I can guide you on how to find the current exchange rate.\n\n1. **Check a Financial Website**: Visit a reliable financial website like XE.com, OANDA, or a similar service.\n2. **Use a Currency Conversion App**: There are many apps available for both iOS and Android that provide real-time exchange rates.\n3. **Bank or Financial Institution**: Your bank or a financial institution can provide the current exchange rate.\n4. **Google**: Simply search "1000 PKR to USD" on Google, and it will show the current exchange rate.\n\nOnce you have the current exchange rate, you can calculate the conversion as follows:\n\\[ \\text{Amount in USD} = \\text{Amount in PKR} \\times \\text{Exchange Rate} \\]\n\nPlease provide the current exchange rate if you know it, and I can do the calculation for you.'

## 2. Using OpenAI Agents SDK

In [ ]:
import asyncio
from openai import AsyncOpenAI # chat completions
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

set_tracing_disabled(disabled=True) # Open AI Tracing == Disable

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)


async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="LahoreTA",
        instructions="You only respond in english.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent, # starting agent
        "What is your name?.", # request
    )
    print(result.final_output)


asyncio.run(main())

I don't have a name. I'm a text-based AI model and I don't have personal experiences, feelings, or a name. I'm here to provide information and answer questions to the best of my ability. You can give me a name if it helps you feel more comfortable talking to me, but I don't have a predetermined name.


In [ ]:
response.json()

{'id': 'gen-1744911192-rye3MFpXN4v0nbdLeI5V',
 'provider': 'DeepInfra',
 'model': 'mistralai/mistral-7b-instruct',
 'object': 'chat.completion',
 'created': 1744911192,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': " Hello! How can I help you today? I am here to answer any question you might have, or help with any information you need. Let's get started! What would you like to know? ♥️💕",
    'refusal': None,
    'reasoning': None}}],
 'usage': {'prompt_tokens': 8, 'completion_tokens': 45, 'total_tokens': 53}}